In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec 29 14:42:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!gdown --id 1YuzCW6s56tV409JRXaAMKFdIkK9xIACw --output "data.zip"
!unzip -q "data.zip"

Downloading...
From: https://drive.google.com/uc?id=1YuzCW6s56tV409JRXaAMKFdIkK9xIACw
To: /content/data.zip
100% 1.13G/1.13G [00:04<00:00, 231MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! pip install byol-pytorch
! pip install transformers

     |████████████████████████████████| 3.4 MB 13.7 MB/s 
     |████████████████████████████████| 895 kB 66.3 MB/s 
     |████████████████████████████████| 596 kB 72.6 MB/s 
     |████████████████████████████████| 61 kB 715 kB/s 
     |████████████████████████████████| 3.3 MB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! cp -r /content/drive/MyDrive/DLCV/HW4/improved-net4.pt /content/
# ! cp -r /content/drive/MyDrive/DLCV/HW4/checkpoint_p2/model2511.pth /content

In [ ]:
import random

import torch
import numpy as np


def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(1126)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler
from torch import optim

import csv
import random
import numpy as np
import pandas as pd

from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

import torch
# from byol_pytorch import BYOL
from torchvision import models

# mini-Imagenet dataset
class OfficeDataset(Dataset):
    def __init__(self, csv_path, data_dir, transforms):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_path).set_index("id")
        self.labels = list(sorted(set(pd.read_csv(csv_path).iloc[:,2])))

        # self.transform = transforms.Compose([
        #     filenameToPILImage,
        #     transforms.ToTensor(),
        #     transforms.Resize((128, 128)),
        #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #     ])
        self.transform = transforms
        
        self.label_dict = {}
        for id, lab in enumerate(self.labels):
          self.label_dict[lab] = id
        
        

    def __getitem__(self, index):
        path = self.data_df.loc[index, "filename"]
        label_str = self.data_df.loc[index, "label"]
        image = self.transform(os.path.join(self.data_dir, path))
        label = self.label_dict[label_str]
        return image, label

    def __len__(self):
        return len(self.data_df)

In [ ]:
train_transform = transforms.Compose([
        filenameToPILImage,
        transforms.AutoAugment(),
        transforms.ToTensor(),
        
        transforms.Resize((128, 128)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

val_transform = transforms.Compose([
        filenameToPILImage,
        transforms.ToTensor(),
        transforms.Resize((128, 128)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])


In [ ]:
test_set = OfficeDataset('/content/hw4_data/office/val.csv', '/content/hw4_data/office/val', transforms = val_transform)
train_set = OfficeDataset('/content/hw4_data/office/train.csv', '/content/hw4_data/office/train', transforms = train_transform)

In [ ]:
print(train_set.label_dict)
print(test_set.label_dict)

{'Alarm_Clock': 0, 'Backpack': 1, 'Batteries': 2, 'Bed': 3, 'Bike': 4, 'Bottle': 5, 'Bucket': 6, 'Calculator': 7, 'Calendar': 8, 'Candles': 9, 'Chair': 10, 'Clipboards': 11, 'Computer': 12, 'Couch': 13, 'Curtains': 14, 'Desk_Lamp': 15, 'Drill': 16, 'Eraser': 17, 'Exit_Sign': 18, 'Fan': 19, 'File_Cabinet': 20, 'Flipflops': 21, 'Flowers': 22, 'Folder': 23, 'Fork': 24, 'Glasses': 25, 'Hammer': 26, 'Helmet': 27, 'Kettle': 28, 'Keyboard': 29, 'Knives': 30, 'Lamp_Shade': 31, 'Laptop': 32, 'Marker': 33, 'Monitor': 34, 'Mop': 35, 'Mouse': 36, 'Mug': 37, 'Notebook': 38, 'Oven': 39, 'Pan': 40, 'Paper_Clip': 41, 'Pen': 42, 'Pencil': 43, 'Postit_Notes': 44, 'Printer': 45, 'Push_Pin': 46, 'Radio': 47, 'Refrigerator': 48, 'Ruler': 49, 'Scissors': 50, 'Screwdriver': 51, 'Shelf': 52, 'Sink': 53, 'Sneakers': 54, 'Soda': 55, 'Speaker': 56, 'Spoon': 57, 'TV': 58, 'Table': 59, 'Telephone': 60, 'ToothBrush': 61, 'Toys': 62, 'Trash_Can': 63, 'Webcam': 64}
{'Alarm_Clock': 0, 'Backpack': 1, 'Batteries': 2, 'B

In [ ]:
a, b = train_set[100]
print(a.shape,b)

torch.Size([3, 128, 128]) 59


In [ ]:
batch_size = 128

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import torchvision.models as models


class Net(nn.Module):
    def __init__(self):
      # TODO
      super(Net, self).__init__()

      # self.load_model = models.wide_resnet50_2(pretrained = True, progress=True)
      self.resnet = models.resnet50(pretrained=False).to('cuda')
      self.resnet.load_state_dict(torch.load('/content/hw4_data/pretrain_model_SL.pt'))
      # print(self.load_model.image_size)
      # self.classifier = nn.Linear(1000, 65)
      # self.classifier = nn.Sequential(
      #       nn.Linear(1000, 80),
      #       nn.ReLU(),
      #       nn.Dropout(),
      #       nn.Linear(80, 80),
      #       nn.ReLU(),
      #       nn.Dropout(),
      #       nn.Linear(80, 65)
      # )
      
      

    def forward(self, x):
      # TODO
      x = self.resnet(x)
      # x = self.classifier(x)
      return x

class Classifier(nn.Module):
    def __init__(self):
      # TODO
      super(Classifier, self).__init__()

      # self.load_model = models.wide_resnet50_2(pretrained = True, progress=True)
      # self.resnet = models.resnet50(pretrained=False).to('cuda')
      # self.resnet.load_state_dict(torch.load('/content/improved-net4.pt'))
      # print(self.load_model.image_size)
      self.classifier = nn.Linear(1000, 65)
      # self.classifier = nn.Sequential(
      #       nn.Linear(1000, 80),
      #       nn.ReLU(),
      #       nn.Dropout(),
      #       nn.Linear(80, 80),
      #       nn.ReLU(),
      #       nn.Dropout(),
      #       nn.Linear(80, 65)
      # )
      
      

    def forward(self, x):
      # TODO
      # x = self.resnet(x)
      x = self.classifier(x)
      return x

In [ ]:
model = Net().to('cuda')
classifier = Classifier().to('cuda')

In [ ]:
from transformers import get_linear_schedule_with_warmup, get_constant_schedule, get_cosine_schedule_with_warmup

def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

def train_save(model, epoch, save_interval, log_interval=31):
    # optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-4)(adam 3e-5 cosine)
    # optimizer = optim.SGD(model.parameters(), lr = 0.0002, momentum = 0.9)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
    criterion = nn.CrossEntropyLoss()
    # load_checkpoint('/content/drive/MyDrive/DLCV/HW4/checkpoint_p2/model2511.pth', model, optimizer)

    model.train()

    # max_steps = len(train_loader) / batch_size
    # scheduler = get_linear_schedule_with_warmup(optimizer, 5, 30)
    # scheduler = get_constant_schedule(optimizer)
    scheduler = get_cosine_schedule_with_warmup(optimizer, 20, 100)
    
    iteration = 0
    for ep in range(epoch):
        # if ep%5 == 0:
        model.train()
        scheduler.step()
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to('cuda'), target.to('cuda')
            optimizer.zero_grad()
            output = model(data).squeeze(0)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            # if iteration % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Lr: {}'.format(
                ep, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), optimizer.param_groups[0]['lr']))
            if iteration % save_interval == 0 and iteration > 0:
                validate(model)
                save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)
            iteration += 1
        validate(model)
        save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)



In [ ]:
def train_save_fix(model, classifier, epoch, save_interval, log_interval=31):
    # optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-4)(adam 3e-5 cosine)
    # optimizer = optim.SGD(model.parameters(), lr = 0.0002, momentum = 0.9)
    optimizer = torch.optim.AdamW(classifier.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
    criterion = nn.CrossEntropyLoss()
    # load_checkpoint('/content/drive/MyDrive/DLCV/HW4/checkpoint_p2/model2511.pth', model, optimizer)

    classifier.train()

    # max_steps = len(train_loader) / batch_size
    # scheduler = get_linear_schedule_with_warmup(optimizer, 5, 30)
    # scheduler = get_constant_schedule(optimizer)
    scheduler = get_cosine_schedule_with_warmup(optimizer, 20, 100)
    
    iteration = 0
    for ep in range(epoch):
        # if ep%5 == 0:
        classifier.train()
        scheduler.step()
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to('cuda'), target.to('cuda')
            optimizer.zero_grad()
            output = model(data)
            output = classifier(output).squeeze(0)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            # if iteration % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Lr: {}'.format(
                ep, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), optimizer.param_groups[0]['lr']))
            if iteration % save_interval == 0 and iteration > 0:
                validate_fix(model, classifier)
                # save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)
            iteration += 1
        validate_fix(model, classifier)
        # save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)

def validate_fix(model, classifier):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    classifier.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to('cuda'), target.to('cuda')
            output = model(data)
            output = classifier(output)
            # print(output.shape, target.shape)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [ ]:
! mkdir -p /content/checkpoint

In [ ]:
train_save_fix(model, classifier, 100, 100)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

Train Epoch: 0 [0/3951 (0%)]	Loss: 4.184492 Lr: 1.0000000000000002e-06
Train Epoch: 0 [128/3951 (3%)]	Loss: 4.181221 Lr: 1.0000000000000002e-06
Train Epoch: 0 [256/3951 (6%)]	Loss: 4.225677 Lr: 1.0000000000000002e-06
Train Epoch: 0 [384/3951 (10%)]	Loss: 4.247908 Lr: 1.0000000000000002e-06
Train Epoch: 0 [512/3951 (13%)]	Loss: 4.239679 Lr: 1.0000000000000002e-06
Train Epoch: 0 [640/3951 (16%)]	Loss: 4.193351 Lr: 1.0000000000000002e-06
Train Epoch: 0 [768/3951 (19%)]	Loss: 4.246045 Lr: 1.0000000000000002e-06
Train Epoch: 0 [896/3951 (23%)]	Loss: 4.203335 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1024/3951 (26%)]	Loss: 4.191077 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1152/3951 (29%)]	Loss: 4.231047 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1280/3951 (32%)]	Loss: 4.193359 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1408/3951 (35%)]	Loss: 4.207572 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1536/3951 (39%)]	Loss: 4.187987 Lr: 1.0000000000000002e-06
Train Epoch: 0 [1664/3951 (42%)]	Loss:

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
model = Net().to('cuda')
state = torch.load('/content/checkpoint/model4433.pth')
model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

In [ ]:
validate(model) #try #2232 #1178 #2387         
#2046 2790
#36 2511 1922 1426 1457 1829 2356
#35 2700 2666 2604 2604 2500 2294 2232 1085 1674 1767 1800 2015 2821

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0556, Accuracy: 130/406 (32%)



In [ ]:
! mkdir -p /content/checkpoint_p2

In [ ]:
! cp -r /content/checkpoint/model2511.pth /content/checkpoint_p2
! cp -r /content/checkpoint/model1922.pth /content/checkpoint_p2
! cp -r /content/checkpoint/model1426.pth /content/checkpoint_p2
! cp -r /content/checkpoint/model1457.pth /content/checkpoint_p2
! cp -r /content/checkpoint/model1829.pth /content/checkpoint_p2
! cp -r /content/checkpoint/model2356.pth /content/checkpoint_p2

In [ ]:
! cp -r /content/checkpoint/model2046.pth /content/drive/MyDrive/DLCV/HW4
! cp -r /content/checkpoint/model2790.pth /content/drive/MyDrive/DLCV/HW4

In [ ]:
! cp -r /content/drive/MyDrive/DLCV/HW4/model2790.pth /content/checkpoint_p2

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
model = Net().to('cuda')
state = torch.load('/content/checkpoint_p2/model2790.pth')
model.load_state_dict(state['state_dict'])

validate(model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0235, Accuracy: 152/406 (37%)



In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)

model1 = Net().to('cuda')
model2 = Net().to('cuda')
model3 = Net().to('cuda')
model4 = Net().to('cuda')
model5 = Net().to('cuda')
model6 = Net().to('cuda')

state1 = torch.load('/content/checkpoint_p2/model2511.pth')
state2 = torch.load('/content/checkpoint_p2/model1922.pth')
state3 = torch.load('/content/checkpoint_p2/model1426.pth')
state4 = torch.load('/content/checkpoint_p2/model1457.pth')
state5 = torch.load('/content/checkpoint_p2/model1829.pth')
state6 = torch.load('/content/checkpoint_p2/model2356.pth')

model1.load_state_dict(state1['state_dict'])
model2.load_state_dict(state2['state_dict'])
model3.load_state_dict(state3['state_dict'])
model4.load_state_dict(state4['state_dict'])
model5.load_state_dict(state5['state_dict'])
model6.load_state_dict(state6['state_dict'])

# optimizer.load_state_dict(state1['optimizer'])

<All keys matched successfully>

In [ ]:
from torchvision.transforms.transforms import ToTensor

criterion = nn.CrossEntropyLoss()
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in valid_loader:
        data, target = data.to('cuda'), target.to('cuda')
        output1 = model1(data)
        output2 = model2(data)
        output3 = model3(data)
        output4 = model4(data)
        output5 = model5(data)
        output6 = model6(data)
        # print(output.shape, target.shape)
        test_loss += criterion(output1, target).item()
        pred1 = output1.max(1, keepdim=True)[1]
        pred2 = output2.max(1, keepdim=True)[1]
        pred3 = output3.max(1, keepdim=True)[1]
        pred4 = output4.max(1, keepdim=True)[1]
        pred5 = output5.max(1, keepdim=True)[1]
        pred6 = output6.max(1, keepdim=True)[1]
        # print(pred1.shape)
        temp = torch.cat((pred1, pred2 ,pred3, pred3, pred3, pred3, pred4, pred5, pred6), 1)
        # print(temp.shape)
        pred_list = []
        
        for id, i in enumerate(temp):
          i = i.to('cpu').numpy()
          # print(i, np.bincount(i).argmax())
          pred_list.append(np.bincount(i).argmax())
          pred = torch.tensor(pred_list).to('cuda')
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(valid_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_loader.dataset),
    100. * correct / len(valid_loader.dataset)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0322, Accuracy: 148/406 (36%)



In [ ]:
validate(model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0362, Accuracy: 145/406 (36%)

